### Terminology point

1. Qubit => _real physical_ entities for developing quantum computers (ex. ion di medan elektrik, Josephson junctions on an ASIC)
2. The State of qubit => _measurable property_ dari qubit (ex. energy level of an electron), the state of two or more qubits is defined as their tensor product
3. Greek symbol ($\ket{\psi}$) => _state space_ dari satu atau lebih qubit
4. $\ket{0}$ and $\ket{1}$ => _basis states of vectors_ => bentuk orthogonal sets dari vektor dengan dimensi n yang dapat membentuk semua vektor
5. _basis vectors_ is orthonormal (linear independent dan have a modulus of 1)
6. [**Born Rule**] $\ket{\psi} = \alpha\ket{0} + \beta\ket{1}$ => superposition state : state of a qubit is a linear combination of orthonormal basis states, 
7. $\alpha$ and $\beta$ are complex number => _probability amplitudes_ dengan mengikuti persamaan <br>
$|\alpha|^2+|\beta|^2=1$
8. tipe data qubit adalah sebuah array dari 2 bilangan kompleks pada $\alpha$ dan $\beta$ dan membuat_state_ dari array tersebut

In [22]:
# membuat qubit
from typing import Optional
import numpy as np

# membuat class state 
class state:
    def __init__(self, vector: np.ndarray):
        # Menyimpan vektor numpy [alpha, beta]
        self.vector = vector

    def __repr__(self):
        # Ini hanya untuk membuatnya tercetak dengan baik
        return f"Qubit State(vector={self.vector})"


def qubit(alpha : Optional[np.complexfloating] = None, beta : Optional[np.complexfloating] = None) -> state:
    """type Optional digunakan karena qc membutuhkan alpha atau/dan beta"""

    if alpha is None and beta is None:
        raise ValueError('alpha, or beta, or both are required')

    if beta is None:
        beta = np.sqrt(1.0 - np.conj(alpha) * alpha)

    if alpha is None:
        alpha = np.sqrt(1.0 - np.conj(beta) * beta)
    
    if not np.isclose(np.conj(alpha) * alpha + np.conj(beta) * beta, 1.0) :
        raise ValueError('Qubit probabilities do not add to 1.')

    qb = np.zeros(2, dtype=np.complex128)
    qb[0] = alpha
    qb[1] = beta
    return state(qb)

print(f"state is just a complex vector ")
print(f"[1,0]T dan [0,1]T adalah computational basis")

# contoh menampilkan status |0>, dan |1>
state_0 = qubit(alpha=1.0)
state_1 = qubit(beta=1.0)
print(f"status |0>: {state_0}")
print(f"status |1>: {state_1}")

#  status superposisi
state_superposition = qubit(alpha=3/5)
print(f"status superposisi : {state_superposition}")


state is just a complex vector 
[1,0]T dan [0,1]T adalah computational basis
status |0>: Qubit State(vector=[1.+0.j 0.+0.j])
status |1>: Qubit State(vector=[0.+0.j 1.+0.j])
status superposisi : Qubit State(vector=[0.6+0.j 0.8+0.j])


### States for qubit
State <= Tensor class (inherite class)<br>
State dari dua atau lebih qubit di definisikan sebagai tensor product, _the state for n qubit is a Tensor of $2^n$ complex numbers, the probability amplitudes_<br><br>
(berapa banyak amplitudo probablitas yang diperlukan untuk mendefinisikan keadaan sistem n-qubit) <br>
(untuk menunjukkan 8 state kita butuh n = 3. sehingga mendapatkan $2^n$ complex number)

sehingga jumlah qubit yang diperlukan untuk mencapai panjang tensor dituliskan dalam $n = log_2(L)$ <= kita bisa menambahkan qubit pada suatu sistem dengan melakukan tensor product dengan berdasar pada persamaan tersebut

In [ ]:
class State(tensor.Tensor) :
    """class State represent single and multi-qubit states"""

    def __repr__(self) -> str:
        s = 'State('
        s += ')'
        return s
    
    def __str__(self) -> str:
        s = f'(self.nbits)-qubit state.'
        s += 'Tensor:\n'
        s += super().__str__()
        return s

Dalam OOP python, kita bisa menghitung berapa jumlah qubit yang diperlukan dengan menggunakan @properti bernama nbits <br><br>
penempatan property nbits ini lebih baik diletakkan pada base_class (Tensor) supaya kelas lain seperti state dan operator dapat mewarisi property nbits (otomatis)

In [ ]:
import math
@property
def nbits(self) -> int:
    """Compute the number of qubits in the states"""

    return int(math.log2(self.shape[0]))

#### Menggabungkan sistem qubit
- 1 Qubit <br>
1. memiliki 2 keadaan basis $\ket 0$ dan $\ket 1$ <br>
2. superposisi state : $\ket \psi = \alpha \ket 0 + \beta \ket 1$<br>
3. vektor basisnya adalah 2 x 1 :$\begin{bmatrix} 1 0  \\ 0 1\end{bmatrix}$ -- normalized vector
4. Total probabilitas = 1 (norm) : $|\alpha|^2 + |\beta|^2 = 1$

- N Qubit <br>
1. memiliki $2^N$ keadaan basis
2. superposisi state : $\ket \psi = c_0 \ket \psi_0 + c_1 \ket \psi_1 + c_2 \ket \psi_2 + \dots + c_N \ket \psi_N $
3. Total Probabilitas = 1 : <br>
semisal terdapat qubit $\bra \psi \ket \psi$. => The amplitude are complex numbers, hence <br>
\begin{align*}
\langle\psi|\psi\rangle &= c_0^* \langle\psi_0| c_0 |\psi_0\rangle + c_1^* \langle\psi_1| c_1 |\psi_1\rangle + \dots + c_n^* \langle\psi_{n-1}| c_n |\psi_{n-1}\rangle \\
&= c_0^* c_0 \langle\psi_0|\psi_0\rangle + c_1^* c_1 \langle\psi_1|\psi_1\rangle + \dots + c_n^* c_n \langle\psi_{n-1}|\psi_{n-1}\rangle \\
&= c_0^* c_0 + c_1^* c_1 + \dots + c_{n-1}^* c_{n-1} \\
&= 1.0
\end{align*}

In [2]:
import random

p1 = state.qubit(alpha=random.random())
x1 = state.qubit(alpha=random.random())
psi = p1 * x1

    # inner product of full state
self.assertTrue(np.allclose(np.inner(psi.conj(),psi), 1.0))

    # inner product of the constituents multiplied
self.assertTrue(np.allclose(np.inner(p1.conj(),p1)*
                            np.inner(x1.conj(),x1), 1.0))

NameError: name 'state' is not defined